In [1]:
import os
import numpy as np
import pandas as pd
import gc
from multiprocessing import Pool
from glob import glob
import utils
import EDA
#sorted(glob('../data/*'))

In [2]:
# setting
month_limit = 12 # max: 96

month_round = 1

In [5]:
cre = utils.read_pickles('../data/credit_card_balance')

cre.drop('SK_ID_PREV', axis=1, inplace=True)
cre = cre[cre['MONTHS_BALANCE']>=-month_limit]

cre['month_round'] = (cre['MONTHS_BALANCE'] / month_round).map(np.floor)
cre.drop('MONTHS_BALANCE', axis=1, inplace=True)

# groupby other credit cards
gr = cre.groupby(['SK_ID_CURR', 'month_round'])
cre_ = gr.size()
cre_.name = 'cc_size'
cre_ = pd.concat([cre_, gr.sum()], axis=1).reset_index() # TODO:NAME_CONTRACT_STATUS
cre_.sort_values(['SK_ID_CURR', 'month_round'], ascending=[True, False], inplace=True)

cre_['AMT_BALANCE-by-AMT_CREDIT_LIMIT_ACTUAL']                   = cre_['AMT_BALANCE'] / cre_['AMT_CREDIT_LIMIT_ACTUAL']
cre_['AMT_DRAWINGS_ATM_CURRENT-by-CNT_DRAWINGS_ATM_CURRENT']     = cre_['AMT_DRAWINGS_ATM_CURRENT'] / cre_['CNT_DRAWINGS_ATM_CURRENT']
cre_['AMT_DRAWINGS_CURRENT-by-CNT_DRAWINGS_CURRENT']             = cre_['AMT_DRAWINGS_CURRENT'] / cre_['CNT_DRAWINGS_CURRENT']
cre_['AMT_DRAWINGS_OTHER_CURRENT-by-CNT_DRAWINGS_OTHER_CURRENT'] = cre_['AMT_DRAWINGS_OTHER_CURRENT'] / cre_['CNT_DRAWINGS_OTHER_CURRENT']
cre_['AMT_DRAWINGS_POS_CURRENT-by-CNT_DRAWINGS_POS_CURRENT']     = cre_['AMT_DRAWINGS_POS_CURRENT'] / cre_['CNT_DRAWINGS_POS_CURRENT']


100%|██████████| 20/20 [00:00<00:00, 25.55it/s]


In [6]:
ids = cre_.SK_ID_CURR.unique()

In [19]:
tmp = cre_[cre_['SK_ID_CURR']==np.random.choice(ids)]
tmp

,SK_ID_CURR,month_round,cc_size,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,AMT_BALANCE-by-AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT-by-CNT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT-by-CNT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT-by-CNT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT-by-CNT_DRAWINGS_POS_CURRENT
776522,361055,-2.0,1,928202.310,900000,0.0,51223.140,0.0,51223.140,45289.665,...,0.0,11.0,15.0,0,0,1.031336,NaN,4656.649091,NaN,4656.649091
776521,361055,-3.0,1,909104.625,900000,90000.0,402859.755,0.0,312859.755,25724.475,...,0.0,8.0,14.0,0,0,1.010116,90000.0,44762.195000,NaN,39107.469375
776520,361055,-4.0,1,517361.310,900000,0.0,0.000,0.0,0.000,31114.845,...,0.0,0.0,13.0,0,0,0.574846,NaN,NaN,NaN,NaN
776519,361055,-5.0,1,535086.225,900000,0.0,0.000,0.0,0.000,46346.805,...,0.0,0.0,12.0,0,0,0.594540,NaN,NaN,NaN,NaN
776518,361055,-6.0,1,931998.060,900000,13500.0,21547.980,0.0,8047.980,46033.200,...,0.0,5.0,11.0,0,0,1.035553,13500.0,3591.330000,NaN,1609.596000
776517,361055,-7.0,1,880063.470,900000,0.0,65626.380,0.0,65626.380,43587.135,...,0.0,22.0,10.0,0,0,0.977848,NaN,2983.017273,NaN,2983.017273
776516,361055,-8.0,1,881217.765,900000,0.0,14257.575,0.0,14257.575,46270.035,...,0.0,6.0,9.0,0,0,0.979131,NaN,2376.262500,NaN,2376.262500
776515,361055,-9.0,1,928273.275,900000,0.0,8761.680,0.0,8761.680,46294.200,...,0.0,5.0,8.0,0,0,1.031415,NaN,1752.336000,NaN,1752.336000
776514,361055,-10.0,1,939101.310,900000,31500.0,101728.080,0.0,70228.080,40284.855,...,0.0,23.0,7.0,0,0,1.043446,31500.0,4238.670000,NaN,3053.394783
776513,361055,-11.0,1,857778.885,900000,18000.0,482558.175,0.0,464558.175,2250.000,...,0.0,29.0,6.0,0,0,0.953088,18000.0,16085.272500,NaN,16019.247414


In [20]:
tmp.AMT_BALANCE.pct_change(-2)

776522    0.794108
776521    0.698987
776520   -0.444890
776519   -0.391991
776518    0.057625
776517   -0.051935
776516   -0.061637
776515    0.082182
776514    0.898924
776513         NaN
776512         NaN
Name: AMT_BALANCE, dtype: float64

In [21]:
tmp['AMT_BALANCE-by-AMT_CREDIT_LIMIT_ACTUAL'].pct_change(-2)

776522    0.794108
776521    0.698987
776520   -0.444890
776519   -0.391991
776518    0.057625
776517   -0.051935
776516   -0.061637
776515    0.082182
776514    0.898924
776513         NaN
776512         NaN
Name: AMT_BALANCE-by-AMT_CREDIT_LIMIT_ACTUAL, dtype: float64